In [2]:
import pandas as pd
import numpy as np

import ames as ames

In [3]:
df_train = pd.read_csv('./datasets/train.csv')
df_test = pd.read_csv('./datasets/test.csv')

In [4]:
df_train_renamed = ames.rename(df_train)
df_test_renamed = ames.rename(df_test)

In [5]:
df_train_cleaned = ames.map(df_train_renamed)
df_test_cleaned = ames.map(df_test_renamed)

In [7]:
df_test_cleaned.to_csv('./datasets/test_cleaned.csv', index=False)
df_train_cleaned.to_csv('./datasets/train_cleaned.csv', index=False)